In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sts
from scipy.optimize import root

 ### Variables Setting

In [2]:
S = 100
K = 95
T = 0.5
div = 0
sigma = 0.2
r = 0.03
N = 5 ## Binomial Tree Node Number
Sim_num = 10000 ## Monte Carlo Simulation Number

 # Plain Vanila

 ## 1. Black Scholes Model

In [3]:
def BS_call(S, K, T, r, sigma, div = 0):
    d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = (S * np.exp(-div * T) * sts.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * sts.norm.cdf(d2, 0.0, 1.0))
    return call

def BS_put(S, K, T, r, sigma, div = 0):
    d1 = (np.log(S / K) + (r -div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = (K * np.exp(-r * T) * sts.norm.cdf(-d2, 0.0, 1.0) - S * np.exp(-div * T ) *sts.norm.cdf(-d1, 0.0, 1.0))
    return put

In [4]:
call = BS_call( S, K, T, r , sigma,div)
call

9.251090315022012

In [5]:
put = BS_put(S,K,T,r,sigma,div)
put

2.836724577312957

 ## 2. Monte Carlo Simulation

$$
\begin{array}{c}
 S_{T} = S_{0} e^{(r-\frac{\sigma^2}{2})T + \sigma \sqrt{T} \epsilon}
\end{array}
$$
\
$$
\begin{array}{c}
 C = e^{-rT} \hat{E}[ \ max(S_T - K , 0 )]
\end{array}
$$

In [6]:
def MC_call(S, K, T, r, sigma , div = 0, Sim_num = 10000) :
    e = np.random.normal(0,1,size = Sim_num)
    simul_S = S * np.exp((r-div-0.5*sigma**2)*T + sigma * np.sqrt(T) * e)
    Payoff = np.maximum(simul_S - K ,0)
    Discount_Payoff = np.exp(-r * T) * Payoff
    C = Discount_Payoff.mean()
    return C

def MC_put(S, K, T, r, sigma , div = 0, Sim_num = 10000) :
    e = np.random.normal(0,1,size = Sim_num)
    simul_S = S * np.exp((r-div-0.5*sigma**2)*T + sigma * np.sqrt(T) * e)
    Payoff = np.maximum( K-simul_S ,0)
    Discount_Payoff = np.exp(-r * T) * Payoff
    P = Discount_Payoff.mean()
    return P


In [7]:
call2 = MC_call(S , K, T, r , sigma, div, Sim_num)
call2

9.242148631354668

In [8]:
put2 = MC_put(S , K, T, r , sigma, div, Sim_num)
put2

2.7575497535780995

 ## 3. Binomial Option Pricing

$$
\begin{array}{c}
 u = e^{\sigma \sqrt{\Delta t}} \ , \ d = \frac{1}{u} , \ p = \frac{e^{(r-\delta ) \Delta t} - d}{u-d}
\end{array}
$$


In [9]:
def UDP(r,div,sigma, T, N) :
    DeltaT = T/N
    u = np.exp(sigma * np.sqrt(DeltaT))
    d = 1/u
    p = (np.exp((r-div) * DeltaT) - d)/(u-d)
    return u, d, p

In [10]:
u,  d, p = UDP(r, div, sigma, T, N)

In [11]:
def S_Tree(S, N, u, d) :
    Temp = np.ones((N+1,N+1))
    Temp[:,0] = 0
    Temp1 = Temp.cumsum(1)
    Te = Temp.copy()
    Te[0,:] = 0
    num_u = np.triu(Temp1 - Te.cumsum(0))
    num_d = np.triu(Temp1 - num_u)
    Tree = (S * np.triu(u**num_u * d ** num_d))
    return Tree

 ### Make Underlying Tree

In [12]:
pd.DataFrame(S_Tree(S,N, u, d))

0           1           2           3           4           5
0  100.0  106.528839  113.483936  120.893120  128.786037  137.194270
1    0.0   93.871294  100.000000  106.528839  113.483936  120.893120
2    0.0    0.000000   88.118199   93.871294  100.000000  106.528839
3    0.0    0.000000    0.000000   82.717693   88.118199   93.871294
4    0.0    0.000000    0.000000    0.000000   77.648169   82.717693
5    0.0    0.000000    0.000000    0.000000    0.000000   72.889341

 ### Make Payoff Tree

In [13]:
Payoff_Tree = np.maximum(S_Tree(S,N,u,d) - K, 0)[:,-1:]
pd.DataFrame(Payoff_Tree, columns = ['Call_Payoff'])

Call_Payoff
0    42.194270
1    25.893120
2    11.528839
3     0.000000
4     0.000000
5     0.000000

In [14]:
def BN_call(S ,K ,T, r, sigma , div, N) :
    u,  d, p = UDP(r, div, sigma, T, N)
    Payoff_Tree = np.maximum(S_Tree(S,N,u,d) - K, 0)[:,-1:]
    Temp = np.ones((N+1,1))
    Temp[0] = 0
    Number_of_u = Temp.cumsum(0).astype(np.int64)[::-1]
    Weighted_Discount_Payoff = np.exp(-r*T) * ( Payoff_Tree * sts.binom.pmf(Number_of_u,N,p) )
    C = Weighted_Discount_Payoff.sum()    
    return C

def BN_put(S, K, T, r, sigma, div, N) :
    u,  d, p = UDP(r, div, sigma, T, N)
    Payoff_Tree = np.maximum( K - S_Tree(S,N,u,d), 0)[:,-1:]
    Temp = np.ones((N+1,1))
    Temp[0] = 0
    Number_of_u = Temp.cumsum(0).astype(np.int64)[::-1]
    Weighted_Discount_Payoff = np.exp(-r*T) * ( Payoff_Tree * sts.binom.pmf(Number_of_u,N,p) )
    P = Weighted_Discount_Payoff.sum()    
    return P

In [15]:
call3 = BN_call(S,K,T,r,sigma,div,N)
call3

9.186070147270566

In [16]:
put3 = BN_put(S,K,T,r,sigma,div,N)
put3

2.7717044095614995

 # Implied Volatility 

 ## 1. Black Scholes

In [17]:
def BS_call_IV(S,K,T,r,div,C) :
    def fun(x , S, K , T, r, div , P) :   # 찾고자하는 변수를 맨 앞에 셋팅
        return BS_call(S,K,T,r,x,div)- P  # 원래 함수값 - 실제옵션Price = 0 이 되게 함수를 셋팅
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div, C)) #찾고자하는 변수 빼고 args로 셋팅
    return x.x

def BS_put_IV(S,K,T,r,div,P) :
    def fun(x,S,K,T,r,div,Price) :
        return BS_put(S,K,T,r,x,div) - Price
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div, P))
    return x.x

In [18]:
BS_call_IV(S ,K,T,r,div, C = call)

array([0.2])

In [19]:
BS_put_IV(S ,K,T,r,div, P = put)

array([0.2])

 ## 2. Binomial

In [20]:
def BN_call_IV(S,K,T,r,div,C,N) :
    def fun(x , S, K , T, r, div , N,P) :
        return BN_call(S,K,T,r,x,div,N)- P
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div,N, C))
    return x.x

def BN_put_IV(S,K,T,r,div,P,N) :
    def fun(x,S,K,T,r,div,N,Price) :
        return BN_put(S,K,T,r,x,div,N) - Price
    x0 = 0.5
    x = root(fun, x0, args = (S, K, T, r, div,N, P))
    return x.x

In [21]:
BN_call_IV(S ,K,T,r,div, C = call3, N = N)

array([0.2])

In [22]:
BN_put_IV(S ,K,T,r,div, P = put3, N = N)

array([0.2])

 # 연습문제
 
 ### 문제 1-1. 만기의 주가가 K 보다 큰 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 콜옵션의 Black Scholes 가격을 구하는 함수를 만드시오.
 
 ### 문제 1-2. 만기의 주가가 K 보다 작은 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 풋옵션의 Black Scholes 가격을 구하는 함수를 만드시오.

In [23]:
def BS_binary_call(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

def BS_binary_put(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [25]:
bs_binary_c  = BS_binary_call(S,K,T,r,sigma,div) 
bs_binary_c

0.6449574164253773

In [26]:
bs_binary_p = BS_binary_put(S,K,T,r,sigma,div)
bs_binary_p

0.3401545231776853

 ### 문제 2-1. 만기의 주가가 K 보다 큰 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 콜옵션의 이항모형 가격을 구하는 함수를 만드시오.
 
 ### 문제 2-2. 만기의 주가가 K 보다 작은 경우 1을 주고 그렇지 않은 경우 0을 주는 binary 풋옵션의 이항모형 가격을 구하는 함수를 만드시오.

In [27]:
def BN_binary_call(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

def BN_binary_put(S,K,T,r,sigma,div) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [29]:
bn_binary_call = BN_binary_call(S,K,T,r,sigma,div,N = 200)
bn_binary_call

0.6416003497701392

In [30]:
bn_binary_put = BN_binary_put(S,K,T,r,sigma,div,N = 200)
bn_binary_put

0.34351158983292696

  ### 문제 3. 유러피안 배리어 콜옵션(down-and-out, B> K)의 closed form 가격은 다음과 같다. 이를 계산해주는 함수를 만드시오.
  \
$$
\begin{array}{c}
C_{down-and-out} = S e^{-\delta T} N(d_1 ) - K e^{-rT} N(d_2) - (\frac{B}{S})^{1+2(r-\delta)/\sigma^2}  S e^{-\delta T} N(h_1 ) + (\frac{B}{S})^{-1+2(r-\delta )/\sigma^2} Ke^{-rT} N(h_2 )
\\
h_1 = \frac{ ln(\frac{B^2}{KS})+(r-\delta + 0.5 \sigma^2)T }{\sigma \sqrt{T} }
\\
h_2 = h_1 - \sigma \sqrt{T}
\end{array}
$$  

In [31]:
def BS_call_down_and_out(S,K,T,r,sigma,div,Barrier = 0.97) :
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################
    return 

In [33]:
BS_call( S, K, T, r , sigma,div)

9.251090315022012

In [34]:
BS_call_down_and_out(S,K,T,r,sigma,div,Barrier = 0.97)

3.7890911904708773

 ### 문제 4. A 주식과 B 주식은 모두 100원에 거래되며 변동성은 각각 20%, 30% 이며, 수익률의 상관계수는 0.57이다. 만기에 다음과 같은 Payoff를 지급하는 옵션의 가치를 계산하는 함수를 만드시오. 변수는 S1, S2, K, T, r, sigma1, sigma2, corr,div1, div2, simul_num을 받으시오. \begin{array}{c} (S_A ,\ S_B 는 \ 각각 \ 만기의 \ A주식과 \ B주식의 \ 가치이다.) \end{array}
 
$$
\begin{array}{c}
Payoff = max(\sqrt{S_A\bullet S_B} - K , 0)
\end{array}
$$  

In [35]:
def customized_option(S1,S2,K,T,r,sigma1,sigma2,corr,div1,div2,Sim_num = 10000) :    
    #############################
    ##풀이과정을 여기에 쓰세요 ##
    #############################


    #############################    
    return 

In [37]:
customized_option(100,100,95,T,sigma1 = 0.2, sigma2 = 0.3, div1 = 0, div2 = 0, r = r , corr = 0.57)

9.331009514782528

 ### 문제 5. 지금은 T = 0 시점이다. 
 
 ### 다음 파라미터를 기준으로 1년(T=1이 될때까지)동안 주가를 일별 시뮬레이션 하고(dT = 1/250), 
 
 ### 만기가 T = 2시점인 콜옵션의 가격과 델타를 매일매일 계산하시오. (오늘 당일 제외하고)
$$ 
\begin{array}{c}
S_0 = 100,\  \sigma = 0.2,\ r = 0.03 ,\ div = 0 , K = 90 
\end{array}
$$

콜옵션의 델타는 \begin{array}{c}
e^{-div\times T}N(d_1)
\end{array}이다.

주가, 콜옵션가격, 델타를 하나의 데이터프레임으로 나타내시오.

| |S|C|Delta|
|------|---|---|---|
| 1/250|99.911	 |14.923 |0.665 |
| 2/250|98.676 |14.753 |0.6615 |
| 3/250|98.814 |14.830 |0.663 |
| 4/250|99.022 |14.955 |0.666 |

In [38]:
#################
### 풀이과정   ##
#################

 #### 결과창

In [42]:
pd.DataFrame([S,C,Delta],index = ['S','C','Delta']).T

S          C     Delta
0    100.595  16.061206  0.686192
1    102.574  17.428802  0.710218
2    101.002  16.312795  0.691196
3     99.979  15.597885  0.678333
4    100.979  16.268136  0.690866
..       ...        ...       ...
245  134.482  41.694162  0.972924
246  134.125  41.336447  0.972280
247  133.226  40.452651  0.970280
248  132.217  39.464068  0.967839
249  132.459  39.687535  0.968697

[250 rows x 3 columns]

 ### 문제 6. 지금은 T=0으로부터 1일 지난 시점이다.  
 
 ### 현재 주식 10개 매수하고 콜옵션 매도로 델타헤지하려고 한다. T=1시점까지 델타헤징 수익을 계산하시오. 
 
 ### 반복문을 사용해도 좋습니다.
 

In [43]:
#################
### 풀이과정   ##
#################

 #### 결과창

In [46]:
PF

S_Value  Call_Delta  current_number_C  required_number_C  \
0    1005.95    0.686192          0.000000         -14.573177   
1    1025.74    0.710218        -14.573177         -14.080176   
2    1010.02    0.691196        -14.080176         -14.467667   
3     999.79    0.678333        -14.467667         -14.742021   
4    1009.79    0.690866        -14.742021         -14.474592   
..       ...         ...               ...                ...   
245  1344.82    0.972924        -10.279220         -10.278292   
246  1341.25    0.972280        -10.278292         -10.285099   
247  1332.26    0.970280        -10.285099         -10.306303   
248  1322.17    0.967839        -10.306303         -10.332297   
249  1324.59    0.968697        -10.332297         -10.323144   

     required_additional_C  Call_Value        Cash     Profit  
0               -14.573177 -234.062793 -771.887200        NaN  
1                 0.493000 -245.400600 -780.417855  -0.078456  
2                -0.387491 -236.008090 -774.034368  -0.022459  
3                -0.274354 -229.944349 -769.693106   0.152546  
4                 0.267429 -235.474635 -773.982096   0.333269  
..                     ...         ...         ...        ...  
245               0.000928 -428.544767 -884.532489  31.742743  
246              -0.006807 -425.149442 -884.180363  31.920195  
247              -0.021204 -416.917265 -883.251871  32.090863  
248              -0.025994 -407.754456 -882.155392  32.260152  
249               0.009153 -409.700137 -882.448069  32.441794  

[250 rows x 8 columns]

 ### 문제 7. 주식포트폴리오를 델타헤징하는 경우 감마포지션으로 인해 제대로 헤지할 수 없는 경우가 많다. 
 
 ### 따라서 옵션의 감마헤징을 위해 매일매일의 감마를 계산해본다.
 
$$
\begin{array}{c}
Gamma = \frac{e^{-div \ \bullet T}}{S_0 \sigma \sqrt{T}} \bullet \frac{1}{\sqrt{2 \pi}} \bullet e^{\frac{{-d_1}^2}{2}}
\end{array}
$$

In [47]:
#################
### 풀이과정   ##
#################

 ### 문제 8. 주식포트폴리오를 델타헤징하는 경우 감마포지션으로 인해 헤지수익이 음의 값을 가지는 경우가 많다.
 
 $$
 \begin{array}{c}
 {New \ Delta}_t = Delta_{t} + \frac{1}{2} {(\Delta S)}^2 Gamma
 \end{array}
 $$
 
 위의 공식에 따라 새로운 델타헤징을 통해 포트폴리오를 헤징하시오.

In [ ]:
#################
### 풀이과정   ##
#################

In [50]:
PF

S_Value  Call_Delta  current_n_C  required_number_C  \
0    1005.95    0.688000     0.000000         -14.534884   
1    1025.74    0.733361   -14.534884         -13.635853   
2    1010.02    0.706475   -13.635853         -14.154774   
3     999.79    0.684991   -14.154774         -14.598724   
4    1009.79    0.697065   -14.598724         -14.345857   
..       ...         ...          ...                ...   
245  1344.82    0.972926   -10.241969         -10.278269   
246  1341.25    0.972431   -10.278269         -10.283507   
247  1332.26    0.971300   -10.283507         -10.295481   
248  1322.17    0.969224   -10.295481         -10.317536   
249  1324.59    0.968775   -10.317536         -10.322314   

     required_additional_C  Call_Value        Cash     Profit  
0               -14.534884 -233.447765 -772.502200        NaN  
1                 0.899030 -237.656585 -788.109424  -0.026009  
2                -0.518921 -230.903935 -779.581330  -0.465264  
3                -0.443950 -227.709227 -772.594279  -0.513506  
4                 0.252868 -233.380341 -776.646164  -0.236506  
..                     ...         ...         ...        ...  
245              -0.036301 -428.543829 -879.861555  36.414616  
246              -0.005238 -425.083662 -879.574652  36.591685  
247              -0.011974 -416.479507 -879.019914  36.760579  
248              -0.022054 -407.171933 -878.079236  36.918831  
249              -0.004778 -409.667191 -877.819363  37.103446  

[250 rows x 8 columns]